In [3]:
from openeye.oechem import *
from openeye.oegraphsim import *
import os
good_recovery_file = "mdck_goodrecovery.sdf"
bad_recovery_file = "mdck_badrecovery.sdf"
working_directory = "/Users/jfeng1/Datasets/ADME"

good_list = []
mol = OEGraphMol()
ifs = oemolistream()
ifs.open(os.path.join(working_directory,good_recovery_file))
while OEReadMolecule(ifs,mol):
    fp = OEFingerPrint()
    OEMakeFP(fp,mol,OEFPType_Circular)
    good_list.append((OEGraphMol(mol),fp))
ifs.close()

bad_list = []
ifs = oemolistream()
ifs.open(os.path.join(working_directory,bad_recovery_file))
while OEReadMolecule(ifs,mol):
    fp = OEFingerPrint()
    OEMakeFP(fp,mol,OEFPType_Circular)
    bad_list.append((OEGraphMol(mol),fp))
ifs.close()

print len(good_list)
print len(bad_list)

ofs = oemolostream()
ofs.open(os.path.join(working_directory,"goodrecovery_neighbors.sdf"))
for mol,fp in good_list:
    best_tanimoto = 0
    best_nbr = None
    for nbr,fp_nbr in bad_list:
        tanimoto = OETanimoto(fp,fp_nbr)
        if tanimoto > best_tanimoto:
            best_nbr = nbr
            best_tanimoto = tanimoto
    smile = OEMolToSmiles(best_nbr)
    mdck = OEGetSDData(best_nbr,"MDCK PERMEABILITY(MDR1-MDCK);Mean;P. Ratio(B-A/A-B) ((none));(Num)")
    OESetSDData(mol,"Nbr_Smiles",smile)
    OESetSDData(mol,"Nbr_Ratio",mdck)
    OEWriteMolecule(ofs,mol)
ofs.close()

486
351
